# Initial Import

In [1]:
from tqdm import tqdm
import os
import numpy as np
import torch
from torch import optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

from main import backgammon

/Applications/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


pygame 2.5.2 (SDL 2.28.3, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Import Dataset

In [2]:
def load_dataset(file_path):
    # Load CSV using pandas
    df = pd.read_csv(file_path)
    
    # Split into features and labels
    data = df.values
    X = torch.tensor(data[:, :-1], dtype=torch.float32)
    y = torch.tensor(data[:, -1], dtype=torch.float32).unsqueeze(1)  # Ensure shape (N, 1)
    
    return TensorDataset(X, y)

# train_file = os.path.join("..","Data","Deep","BearoffRace",'train.txt')
# validation_file = os.path.join("..","Data","Deep","BearoffRace",'validation.txt')
# test_file = os.path.join("..","Data","Deep","BearoffRace",'test.txt')

# train_dataset = load_dataset(train_file)
# validation_dataset = load_dataset(validation_file)
# test_dataset = load_dataset(test_file)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# val_loader = DataLoader(validation_dataset, batch_size=64)
# test_loader = DataLoader(test_dataset, batch_size=64)

# # Example: iterate over a batch from the train set
# for X_batch, y_batch in train_loader:
#     print("Input shape:", X_batch.shape)
#     print("Labels shape:", y_batch.shape)
#     break

In [3]:
class BGNet(nn.Module):
    def __init__(self):
        super(BGNet, self).__init__()
        # Shared input-to-hidden layer: both board states use the same weights.
        self.fc_shared = nn.Linear(289, 12)
        # Hidden-to-output layer: we use a single layer that will be applied
        # to both hidden representations. In the forward pass we explicitly
        # flip the sign for one branch to enforce that its effect is the negative.
        self.fc_out = nn.Linear(12, 1)

    def forward(self, board_left, board_right):
        # Process left board: apply shared fc layer and ReLU activation.
        h_left = F.relu(self.fc_shared(board_left))
        # Process right board: same shared fc layer.
        h_right = F.relu(self.fc_shared(board_right))
        
        # Compute the output for each branch using the same fc_out layer.
        out_left = self.fc_out(h_left)
        # Explicitly invert the output from the right branch.
        out_right = -self.fc_out(h_right)
        
        # The network's decision is the sum of these contributions.
        # This is equivalent to computing the difference between the two evaluations.
        diff = out_left + out_right
        
        # Apply a sigmoid activation so that final output is between 0 and 1.
        final_output = torch.sigmoid(diff)
        return final_output

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

def evaluate(model, data_loader, device, optimizer, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch[:, :289], X_batch[:, 289:])
            loss = criterion(outputs, y_batch)
            total_loss += loss.item() * y_batch.size(0)

            preds = (outputs > 0.5).float()
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

def train(num_epochs, train_loader, model, criterion, optimizer, val_loader, test_loader, model_type):
    # === Training Loop ===
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss = 0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            board_left = X_batch[:, :289]
            board_right = X_batch[:, 289:]

            outputs = model(board_left, board_right)
            loss = criterion(outputs, y_batch)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * y_batch.size(0)

            # For training accuracy
            preds = (outputs > 0.5).float()
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

        avg_train_loss = running_loss / total
        train_acc = correct / total

        if epoch % 10 == 0:
            val_loss, val_acc = evaluate(model, val_loader, device, optimizer, criterion)
            test_loss, test_acc = evaluate(model, test_loader, device, optimizer, criterion)
            
            print(f"Epoch {epoch:03d}:")
            print(f"  Train Loss: {avg_train_loss:.4f}, Accuracy: {train_acc*100:.2f}%")
            print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc*100:.2f}%")
            print(f"  Test  Loss: {test_loss:.4f}, Accuracy: {test_acc*100:.2f}%\n")

        torch.save(model.state_dict(), f"{model_type}.pth")


In [5]:
# Contact Model

train_file = os.path.join("..","Data","Deep","Contact",'train.txt')
validation_file = os.path.join("..","Data","Deep","Contact",'validation.txt')
test_file = os.path.join("..","Data","Deep","Contact",'test.txt')

train_dataset = load_dataset(train_file)
validation_dataset = load_dataset(validation_file)
test_dataset = load_dataset(test_file)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)
contact_model = BGNet()

# Loss function: Binary Cross-Entropy (good for sigmoid output)
criterion = nn.BCELoss()

# Optimizer
optimizer = optim.Adam(contact_model.parameters(), lr=0.001)

# Training settings
num_epochs = 50  # Change as needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
contact_model.to(device)
train(num_epochs, train_loader, contact_model, criterion, optimizer, val_loader, test_loader, model_type="contact_50")

Epoch 010:
  Train Loss: 0.5167, Accuracy: 74.49%
  Val   Loss: 0.5054, Accuracy: 75.23%
  Test  Loss: 0.6311, Accuracy: 67.68%

Epoch 020:
  Train Loss: 0.4485, Accuracy: 79.43%
  Val   Loss: 0.4379, Accuracy: 80.35%
  Test  Loss: 0.6687, Accuracy: 65.93%

Epoch 030:
  Train Loss: 0.4023, Accuracy: 82.06%
  Val   Loss: 0.3904, Accuracy: 83.18%
  Test  Loss: 0.7090, Accuracy: 65.93%

Epoch 040:
  Train Loss: 0.3681, Accuracy: 83.89%
  Val   Loss: 0.3578, Accuracy: 84.54%
  Test  Loss: 0.7641, Accuracy: 65.34%

Epoch 050:
  Train Loss: 0.3434, Accuracy: 85.41%
  Val   Loss: 0.3326, Accuracy: 86.09%
  Test  Loss: 0.8115, Accuracy: 66.16%



In [6]:
# Midboard Race Model

train_file = os.path.join("..","Data","Deep","MidboardRace",'train.txt')
validation_file = os.path.join("..","Data","Deep","MidboardRace",'validation.txt')
test_file = os.path.join("..","Data","Deep","MidboardRace",'test.txt')

train_dataset = load_dataset(train_file)
validation_dataset = load_dataset(validation_file)
test_dataset = load_dataset(test_file)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)
midboard_race_model = BGNet()

# Loss function: Binary Cross-Entropy (good for sigmoid output)
criterion = nn.BCELoss()

# Optimizer
optimizer = optim.Adam(midboard_race_model.parameters(), lr=0.001)

# Training settings
num_epochs = 50  # Change as needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
midboard_race_model.to(device)
train(num_epochs, train_loader, midboard_race_model, criterion, optimizer, val_loader, test_loader, model_type="midboard_race_50")

Epoch 010:
  Train Loss: 0.4175, Accuracy: 82.17%
  Val   Loss: 0.4014, Accuracy: 82.53%
  Test  Loss: 0.4415, Accuracy: 79.15%

Epoch 020:
  Train Loss: 0.3782, Accuracy: 84.18%
  Val   Loss: 0.3597, Accuracy: 85.15%
  Test  Loss: 0.4182, Accuracy: 80.65%

Epoch 030:
  Train Loss: 0.3608, Accuracy: 85.00%
  Val   Loss: 0.3389, Accuracy: 85.94%
  Test  Loss: 0.4172, Accuracy: 79.90%

Epoch 040:
  Train Loss: 0.3477, Accuracy: 85.49%
  Val   Loss: 0.3234, Accuracy: 87.16%
  Test  Loss: 0.4231, Accuracy: 80.40%

Epoch 050:
  Train Loss: 0.3382, Accuracy: 86.21%
  Val   Loss: 0.3117, Accuracy: 87.86%
  Test  Loss: 0.4280, Accuracy: 79.90%



In [7]:
# Bearoff
train_file = os.path.join("..","Data","Deep","BearoffRace",'train.txt')
validation_file = os.path.join("..","Data","Deep","BearoffRace",'validation.txt')
test_file = os.path.join("..","Data","Deep","BearoffRace",'test.txt')

train_dataset = load_dataset(train_file)
validation_dataset = load_dataset(validation_file)
test_dataset = load_dataset(test_file)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

bearoff_race_model = BGNet()

# Loss function: Binary Cross-Entropy (good for sigmoid output)
criterion = nn.BCELoss()

# Optimizer
optimizer = optim.Adam(bearoff_race_model.parameters(), lr=0.001)

# Training settings
num_epochs = 50  # Change as needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bearoff_race_model.to(device)
train(num_epochs, train_loader, bearoff_race_model, criterion, optimizer, val_loader, test_loader, model_type="bearoff_race_50")

Epoch 010:
  Train Loss: 0.2728, Accuracy: 89.30%
  Val   Loss: 0.2651, Accuracy: 89.74%
  Test  Loss: 0.3014, Accuracy: 86.95%

Epoch 020:
  Train Loss: 0.1918, Accuracy: 92.32%
  Val   Loss: 0.1869, Accuracy: 92.69%
  Test  Loss: 0.2136, Accuracy: 90.70%

Epoch 030:
  Train Loss: 0.1444, Accuracy: 94.52%
  Val   Loss: 0.1415, Accuracy: 94.57%
  Test  Loss: 0.1678, Accuracy: 92.99%

Epoch 040:
  Train Loss: 0.1228, Accuracy: 95.55%
  Val   Loss: 0.1201, Accuracy: 95.51%
  Test  Loss: 0.1416, Accuracy: 93.96%

Epoch 050:
  Train Loss: 0.1115, Accuracy: 95.87%
  Val   Loss: 0.1088, Accuracy: 96.05%
  Test  Loss: 0.1313, Accuracy: 94.62%

